## RAG 구현
- fine tuning 불편함 해결

In [1]:
!pip install chromadb -qqq # vector database

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.6 MB/s eta 0:00:0

In [2]:
import chromadb
chroma_client = chromadb.Client()
chroma_client

### Vector DB에 데이터 넣기

In [3]:
documents = [
    "대한민국의 수도인 서울은 한강을 중심으로 동서로 나뉘며, 다양한 문화와 역사가 공존하는 도시이다. 고궁과 현대식 빌딩이 조화를 이루며, 관광객들에게 많은 볼거리를 제공한다. 또한, 강남과 강북 지역의 차이도 서울의 독특한 특징 중 하나로 꼽힌다.",
    "한반도의 기후는 사계절이 뚜렷하며, 여름에는 북태평양 기단의 영향을 받아 무덥고 습하다. 반면 겨울에는 시베리아 기단의 영향을 받아 춥고 건조한 날씨가 지속된다. 봄과 가을은 짧지만 쾌적한 기후를 자랑하며, 여행하기 좋은 계절이다.",
    "김치는 한국을 대표하는 전통 음식으로, 배추나 무를 소금에 절여 양념을 발라 발효시킨다. 지역마다 다양한 종류가 있으며, 대표적으로 배추김치, 깍두기, 동치미 등이 있다. 한국인들은 김치를 주식과 함께 즐겨 먹으며, 건강에도 좋은 음식으로 알려져 있다.",
    "경복궁은 조선 시대에 건립된 궁궐로, 서울의 중심부에 위치해 있다. 광화문을 지나면 근정전과 경회루 등 다양한 건축물을 볼 수 있으며, 매년 많은 관광객들이 방문한다. 특히, 한복을 입고 방문하면 더욱 특별한 경험을 할 수 있다.",
    "제주도는 대한민국에서 가장 큰 섬으로, 독특한 자연환경과 관광 명소로 유명하다. 한라산을 비롯하여 성산일출봉, 용두암 등의 자연경관이 있으며, 다양한 전통 문화와 먹거리를 즐길 수 있다. 온화한 기후 덕분에 사계절 내내 여행하기 좋다."
]

In [4]:
# Vector data 를 영구성 있게 저장
chroma_client = chromadb.PersistentClient(path = './chroma_db')
chroma_client

In [5]:
from transformers import AutoTokenizer, AutoModel
# 모델 및 토크나이저 로드
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [6]:
## Embedding 과정 
text = documents[0]
inputs = tokenizer(text, return_tensors='pt'
             , truncation = True
             , padding = True
             , max_length = 512) # 사전 맞는 글자 나옴
# inputs

In [7]:
import torch
with torch.no_grad():
    embedding = model(**inputs).last_hidden_state[:, 0, :].squeeze().numpy()
embedding

array([-8.26943964e-02,  2.83078104e-01,  3.51984710e-01, -1.21131599e-01,
       -7.63266534e-02, -2.16000989e-01, -8.87172967e-02, -1.51869670e-01,
        2.39160016e-01, -1.54747680e-01,  3.31681907e-01,  2.06611723e-01,
        1.15314454e-01, -2.29622573e-01,  2.54395396e-01, -1.81165293e-01,
       -1.52802780e-01,  1.28980711e-01,  1.10485397e-01,  3.74626189e-01,
        1.04376696e-01,  1.49214923e-01, -1.98120885e-02, -1.79707617e-01,
       -2.39626229e-01,  1.85221985e-01, -7.01640993e-02, -4.32610400e-02,
        3.11028540e-01, -9.39681351e-01, -2.42199898e-01, -5.87983392e-02,
       -3.17704044e-02,  4.13708538e-02, -1.71108812e-01, -1.78418010e-01,
       -5.14833443e-02,  3.31123352e-01,  4.81579527e-02,  1.76339433e-01,
       -2.13059232e-01, -3.12879741e-01,  1.18538745e-01, -2.39916265e-01,
        9.15384963e-02,  4.53749187e-02, -5.03587574e-02, -3.18319350e-01,
       -2.87917376e-01, -4.31988708e-04, -1.72749668e-01,  1.21962905e-01,
       -1.32388696e-01, -

In [8]:
# Vector data에 data 넣기
collection = chroma_client.get_or_create_collection(name = 'semantic_cache') # Vector DB에 Collection name 부여

doc_id = 90
collection.add(documents = [text]
              , embeddings = [embedding.tolist()]
              , ids = [str(doc_id)] )

In [9]:
def embedded_text(text):
    inputs = tokenizer(text, return_tensors='pt'
                 , truncation = True, padding = True, max_length = 512) # 사전 맞는 글자 나옴
    with torch.no_grad():
        embedding = model(**inputs).last_hidden_state[:, 0, :].squeeze().numpy()
    return embedding.tolist()
    # pass

In [10]:
def add_vectors_to_chroma(text, doc_id):
    collection.add(documents = [text]
              , embeddings = [embedded_text(text)]
              , ids = [str(doc_id)])
    # pass

In [11]:
## documents를 chroma 저장 
for idx, document in enumerate(documents):
    add_vectors_to_chroma(document, idx)
    pass

In [12]:
## chroma 저장 확인
records = collection.get()
records
collection.get(include=['embeddings', 'documents'])

{'ids': ['90', '0', '1', '2', '3', '4'],
 'embeddings': array([[-0.0826944 ,  0.2830781 ,  0.35198471, ...,  0.41575262,
         -0.24117969, -0.27682903],
        [-0.0826944 ,  0.2830781 ,  0.35198471, ...,  0.41575262,
         -0.24117969, -0.27682903],
        [-0.01504656,  0.33184266,  0.45812598, ...,  0.22800539,
         -0.12948731, -0.15413681],
        [-0.10022887,  0.40820131,  0.39269128, ...,  0.21185265,
         -0.17258941, -0.18474115],
        [-0.07150055,  0.4189989 ,  0.47206962, ...,  0.31835684,
         -0.24478322, -0.24589406],
        [-0.1056835 ,  0.43485606,  0.47109622, ...,  0.31788516,
         -0.20172012, -0.17516178]]),
 'documents': ['대한민국의 수도인 서울은 한강을 중심으로 동서로 나뉘며, 다양한 문화와 역사가 공존하는 도시이다. 고궁과 현대식 빌딩이 조화를 이루며, 관광객들에게 많은 볼거리를 제공한다. 또한, 강남과 강북 지역의 차이도 서울의 독특한 특징 중 하나로 꼽힌다.',
  '대한민국의 수도인 서울은 한강을 중심으로 동서로 나뉘며, 다양한 문화와 역사가 공존하는 도시이다. 고궁과 현대식 빌딩이 조화를 이루며, 관광객들에게 많은 볼거리를 제공한다. 또한, 강남과 강북 지역의 차이도 서울의 독특한 특징 중 하나로 꼽힌다.',
  '한반도의 기후는 사계절이 뚜렷하며, 여름에는 북태평양

In [13]:
question = "북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?"
response = collection.query(query_embeddings=[embedded_text(question)], n_results=1)
print(f"질문: {question}\n결과: {response}\n")

질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
결과: {'ids': [['1']], 'embeddings': None, 'documents': [['한반도의 기후는 사계절이 뚜렷하며, 여름에는 북태평양 기단의 영향을 받아 무덥고 습하다. 반면 겨울에는 시베리아 기단의 영향을 받아 춥고 건조한 날씨가 지속된다. 봄과 가을은 짧지만 쾌적한 기후를 자랑하며, 여행하기 좋은 계절이다.']], 'uris': None, 'data': None, 'metadatas': [[None]], 'distances': [[4.638636776690913]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}

